#  OCR (Optical Character Recognition) - Experimento
## Utilização das bibliotecas [opencv](https://opencv.org/) e  [Tesseract OCR](https://tesseract-ocr.github.io/) para o reconhecimento de texto em imagens e da biblioteca [JiWER](https://github.com/jitsi/jiwer) para cálculo de mérticas de perfomance

*   Mais detlalhes sobre  o funcionamento dos algorítimos e das línguas nos quais o mesmo podem são utilizados são encontrados na [Tesseract documentation](https://tesseract-ocr.github.io/tessdoc/Data-Files)

*   Caso seja passado um arquivo .xlsx com as strings de target pode visualizar a perfonrmance do algorítimo



## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
dataset = "/tmp/data/ocr_dataset-2.zip" #@param {type:"string"}
image_path = "input_image" #@param {type:"string",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}
target = "target_OCR" #@param {type:"string",label:"Atributo alvo", description:"Seu modelo tentará prever os valores do alvo."}

#Hyperparams
bbox_conf = 60 #@param {type:"number",label:"Confiabilidade do bbox", description:"O quanto de confiabilidade o algorítmo deve possuir sobre o bbox para que o mesmo apareça."}

#Pytesseact Params
segmentation_mode = "Assume a single uniform block of text."  #@param ["Orientation and script detection (OSD) only.","Automatic page segmentation with OSD.","Automatic page segmentation, but no OSD, or OCR.","Fully automatic page segmentation, but no OSD. (Default)","Assume a single column of text of variable sizes.","Assume a single uniform block of vertically aligned text.","Assume a single uniform block of text.","Treat the image as a single text line.","Treat the image as a single word.","Treat the image as a single word in a circle.","Treat the image as a single character.","Sparse text. Find as much text as possible in no particular order.","Sparse text with OSD","Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific."] {type:"string",label:"Modeo de segmentação do PyTesseract",description:"Para mais informações acesse a documentação linkada no inicio do notebook"}
ocr_engine = "Neural nets LSTM engine only." #@param ["Legacy engine only.","Neural nets LSTM engine only.","Legacy + LSTM engines.","Default, based on what is available."] {type:"string",label:"OCR engine do Pytesseract",description:"Para mais informações acesse a documentação linkada no inicio do notebook"}
language = "por" #@param ["por","eng"] {type:"string",label:"Idioma pré teinado",description:"Para mais informações acesse a documentação linkada no inicio do notebook"}

#Return formart
bbox_return = "np_array" #@param ["np_array","image"] {type:"string",label:"Forma de retorno dos bboxes",description:"Escolher se bboxes serão retornados na imagem ou como um numpy array"}
image_return_format = "N/A" #@param ["N/A",".jpg",".png"] {type:"string",label:"Formato de retorno da imagem caso bbox_return = image",description:"Escolher formato de retorno da imagem, N/A se retornar numpy array"}
remove_linebreaks = True #@param [True,False] {type:"boolean",label:"Remove quebras de linha",description:"Caso True remove \n e \t dos resultados.Vale ressaltar que o texto de referência na tabela .xlsx caso haja, deve considerar este fato para calcular as métricas corretamente"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import os
folder = os.path.join("/tmp/data", "OCR_dataset")

!mkdir -p {folder}
!unzip -o {dataset} -d {folder}

In [ ]:
folder = os.path.join("/tmp/data", "OCR_dataset")

dataset_path = [x for x in os.listdir(folder) if x.endswith(".xlsx") or x.endswith(".csv")]
dataset_path = os.path.join(folder, dataset_path[0])

In [ ]:
import pandas as pd

df = None

if '.xlsx' in dataset_path:
    df = pd.read_excel(dataset_path)
else:
    df = pd.read_csv(dataset_path)


df[image_path] = folder + "/" + df[image_path]

## Remoção de linhas com valores faltantes no atributo alvo
Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.


In [ ]:
df.dropna(inplace=True)

X = df[image_path].to_numpy()
y = df[target].to_numpy()

## Chamada da Classe de OCR

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-ocr/ocr.py

In [ ]:
hyperparams = {'bbox_conf':bbox_conf}
model_parameters = {'ocr_engine':ocr_engine,'segmentation_mode':segmentation_mode,'language':language}
return_formats = {'bbox_return':bbox_return,'image_return_format':image_return_format, 'remove_linebreaks':remove_linebreaks}

In [ ]:
from ocr import Class_Pytesseract_OCR

model = Class_Pytesseract_OCR(hyperparams, model_parameters,return_formats)
model.get_result_dataframe(X, y)

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar as métricas:`MER`, `WER`, `WIL`, `WIP` <br>


In [ ]:
from platiagro import save_metrics

save_metrics(MER=model.avg_mer, WER=model.avg_wer, WIL=model.avg_wil, WIP=model.avg_wip)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [ ]:
from joblib import dump

artifacts = {
    "hyperparams": hyperparams,
    "model_parameters": model_parameters,
    "return_formats":return_formats
}

dump(artifacts, "/tmp/data/ocr.joblib")